In [1]:
# Inicialice app
import os, pyspark
from pyspark.sql import *
#os.chdir("/home/ubuntu")
spark = SparkSession.builder.appName("prueba").master("local[*]").getOrCreate()
sc = spark.sparkContext

In [2]:
# Creo la funcion para parsear las fechas
import datetime
import time

def formatoFecha(elemento):
    if elemento.endswith('Z') and elemento[10]=="T" and elemento[19]==".":
        output = elemento[0:10] + " " + elemento[11:19] + elemento[20:]
        output = output[:-1]
        
    elementoFormateado=time.strptime(output, "%Y-%m-%d %H:%M:%S%f")
    #return output
    return datetime.datetime(elementoFormateado[0],elementoFormateado[1],elementoFormateado[2],elementoFormateado[3],
                            elementoFormateado[4],elementoFormateado[5],elementoFormateado[6]) #.replace(tzinfo=pytz.utc)

In [3]:
# Probamos que funciona bien
import pytz
a="2018-06-25T10:42:56.366000Z"
b=formatoFecha(a).replace(tzinfo=pytz.utc)
b



datetime.datetime(2018, 6, 25, 10, 42, 56, tzinfo=<UTC>)

In [4]:
# Cargamos los datos desde el fichero
data_file = "../Desktop/Data Science/00 -TFM/8- Fichero de prueba/gdax_data_0001.gz"
data = spark.read.json(data_file).sample(False,0.1,1234)

In [5]:
data.count()

1038318

In [6]:
# a=data.limit(100000)
# a.write.mode("overwrite").json("../Desktop/Data Science/00 -TFM/8- Fichero de prueba/10pgdax_data_000110.json")

In [25]:
data.filter()

[Row(_corrupt_record=None, client_oid=None, funds=None, last_trade_id=None, maker_order_id=None, order_id='9bdd240e-b4f6-416c-9b63-a896f35fd45e', order_type=None, price='4707.84000000', product_id='BTC-GBP', reason='canceled', remaining_size='3.31000000', sequence=3241615408, side='buy', size=None, taker_order_id=None, time='2018-06-25T10:42:56.270000Z', trade_id=None, type='done'),
 Row(_corrupt_record=None, client_oid=None, funds=None, last_trade_id=None, maker_order_id=None, order_id='a4edd96f-f8ae-4584-b216-cf8a338aac1a', order_type=None, price='81.03000000', product_id='LTC-USD', reason='canceled', remaining_size='3.47483000', sequence=2272246309, side='buy', size=None, taker_order_id=None, time='2018-06-25T10:42:56.271000Z', trade_id=None, type='done'),
 Row(_corrupt_record=None, client_oid=None, funds=None, last_trade_id=None, maker_order_id=None, order_id='f5e5eeb0-cee5-4d04-b862-ea4d6f69d230', order_type=None, price='398.74000000', product_id='ETH-EUR', reason='canceled', rema

In [7]:
# Creamos la funcion para parsear el RDD:
from pyspark.sql import Row
import datetime

#  ESTA FUNCION RECIBE UN RRD SIN HEADER
def parseaditoBomba(linea):
    numericPrice = linea[7]
    numericSize = linea[13]
    parsedTime = linea[15]

    if linea[13] != None:
        numericSize = float(linea[13])
    if linea[7] != None:
        numericPrice = float(linea[7])
    if linea[15] != None:
        parsedTime = formatoFecha(linea[15])
        
    return(Row(
        client_oid=linea[1],
        funds = linea[2],  
        last_trade_id= linea[3],
        maker_order_id  = linea[4],
        order_id    = linea[5],
        order_type  = linea[6],
        price  = numericPrice,
        product_id  = linea[8],
        reason  = linea[9],
        remaining_size  = linea[10],
        sequence  = linea[11],
        side  = linea[12],
        size  = numericSize,
        taker_order_id  = linea[14],
        time  = parsedTime,
        trade_id  = linea[16],
        type  = linea[17]
    ))

In [8]:
dataParsed = data.rdd.map(lambda x: parseaditoBomba(x))
dataParsed.take(5)

[Row(client_oid=None, funds=None, last_trade_id=None, maker_order_id=None, order_id='9bdd240e-b4f6-416c-9b63-a896f35fd45e', order_type=None, price=4707.84, product_id='BTC-GBP', reason='canceled', remaining_size='3.31000000', sequence=3241615408, side='buy', size=None, taker_order_id=None, time=datetime.datetime(2018, 6, 25, 10, 42, 56), trade_id=None, type='done'),
 Row(client_oid=None, funds=None, last_trade_id=None, maker_order_id=None, order_id='a4edd96f-f8ae-4584-b216-cf8a338aac1a', order_type=None, price=81.03, product_id='LTC-USD', reason='canceled', remaining_size='3.47483000', sequence=2272246309, side='buy', size=None, taker_order_id=None, time=datetime.datetime(2018, 6, 25, 10, 42, 56), trade_id=None, type='done'),
 Row(client_oid=None, funds=None, last_trade_id=None, maker_order_id=None, order_id='f5e5eeb0-cee5-4d04-b862-ea4d6f69d230', order_type=None, price=398.74, product_id='ETH-EUR', reason='canceled', remaining_size='20.79969114', sequence=780099199, side='sell', size=

In [9]:
# Dividimos los diferentes pares

dataBTC_USD=dataParsed.filter(lambda x: x.product_id == "BTC-USD")
dataBTC_EUR=dataParsed.filter(lambda x: x.product_id == "BTC-EUR")
dataBTC_GPB=dataParsed.filter(lambda x: x.product_id == "BTC-GPB")

dataETH_USD=dataParsed.filter(lambda x: x.product_id == "ETH-USD")
dataETH_EUR=dataParsed.filter(lambda x: x.product_id == "ETH-EUR")

dataLTC_USD=dataParsed.filter(lambda x: x.product_id == "LTC-USD")
dataLTC_EUR=dataParsed.filter(lambda x: x.product_id == "LTC-EUR")

dataLTC_BTC=dataParsed.filter(lambda x: x.product_id == "LTC-BTC")
dataETH_BTC=dataParsed.filter(lambda x: x.product_id == "ETH-BTC")

In [27]:
dataBTC_USD.filter(lambda x: x.type == "open").take(35)

[Row(client_oid=None, funds=None, last_trade_id=None, maker_order_id=None, order_id='bc2e234b-814d-43a2-889b-e2bc5df1442f', order_type=None, price=6184.51, product_id='BTC-USD', reason=None, remaining_size='0.25000000', sequence=6180798878, side='buy', size=None, taker_order_id=None, time=datetime.datetime(2018, 6, 25, 10, 42, 56), trade_id=None, type='open'),
 Row(client_oid=None, funds=None, last_trade_id=None, maker_order_id=None, order_id='bdf5a29e-2693-44e1-a642-f7d010615ae7', order_type=None, price=6195.46, product_id='BTC-USD', reason=None, remaining_size='0.04000000', sequence=6180798898, side='sell', size=None, taker_order_id=None, time=datetime.datetime(2018, 6, 25, 10, 42, 56), trade_id=None, type='open'),
 Row(client_oid=None, funds=None, last_trade_id=None, maker_order_id=None, order_id='a626edb5-a655-4475-8259-cf74e1a2132e', order_type=None, price=6195.46, product_id='BTC-USD', reason=None, remaining_size='0.06285586', sequence=6180798985, side='sell', size=None, taker_or

In [10]:
# Sacamos el tamaño de las órdenes matcheadas, su timestamp y creamos un contador.
dataBTC_USD_match=dataBTC_USD.filter(lambda x: x.type == "match" and x.size != None and x.time.day != None).map(lambda x: (x.time, (x.size,1)))
dataBTC_USD_match.take(3)


[(datetime.datetime(2018, 6, 25, 10, 45, 4), (0.00108, 1)),
 (datetime.datetime(2018, 6, 25, 10, 45, 28), (0.00121067, 1)),
 (datetime.datetime(2018, 6, 25, 10, 45, 28), (0.05000727, 1))]

In [11]:
# ReduceByKey para tener para cada timestamp la suma del volumen y el número de matches.
dataBTC_USD_match.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).take(30)

[(datetime.datetime(2018, 6, 25, 10, 45, 4), (0.00108, 1)),
 (datetime.datetime(2018, 6, 25, 10, 45, 28), (0.051217939999999997, 2)),
 (datetime.datetime(2018, 6, 25, 10, 47, 27), (0.2394, 1)),
 (datetime.datetime(2018, 6, 25, 10, 47, 44), (5.09e-06, 1)),
 (datetime.datetime(2018, 6, 25, 10, 48), (0.10248, 2)),
 (datetime.datetime(2018, 6, 25, 10, 48, 42), (1.0, 1)),
 (datetime.datetime(2018, 6, 25, 10, 48, 46), (0.50950307, 2)),
 (datetime.datetime(2018, 6, 25, 10, 50, 37), (0.2448, 1)),
 (datetime.datetime(2018, 6, 25, 10, 51, 10), (0.3724, 1)),
 (datetime.datetime(2018, 6, 25, 10, 54, 53), (0.0494, 1)),
 (datetime.datetime(2018, 6, 25, 10, 55, 4), (0.9916, 1)),
 (datetime.datetime(2018, 6, 25, 10, 55, 10), (0.00127, 1)),
 (datetime.datetime(2018, 6, 25, 10, 55, 34), (0.27798657000000004, 2)),
 (datetime.datetime(2018, 6, 25, 10, 56, 27), (0.35045698, 1)),
 (datetime.datetime(2018, 6, 25, 10, 56, 42), (0.16132961, 1)),
 (datetime.datetime(2018, 6, 25, 10, 57, 41), (0.023, 1)),
 (date

In [12]:
# Añadimos el side y el precio
dataBTC_USD_match=dataBTC_USD.filter(lambda x: x.type == "match" and x.size != None and x.time.day != None).map(lambda x: ((x.time, x.side), (x.size,x.price,1)))
#dataBTC_USD_matchList=dataBTC_USD_match.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1], x[2]+y[2], list(x[3]+y[3]))).collect()

In [15]:
dataBTC_USD_match.max(lambda x: x[1][1])

((datetime.datetime(2018, 6, 25, 16, 6, 16), 'sell'), (0.001, 6321.85, 1))

In [13]:
dataBTC_USD_match.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1], x[2]+y[2])).take(5)

[((datetime.datetime(2018, 6, 25, 10, 45, 4), 'buy'), (0.00108, 6194.11, 1)),
 ((datetime.datetime(2018, 6, 25, 10, 45, 28), 'sell'),
  (0.051217939999999997, 12388.380000000001, 2)),
 ((datetime.datetime(2018, 6, 25, 10, 47, 27), 'sell'), (0.2394, 6193.62, 1)),
 ((datetime.datetime(2018, 6, 25, 10, 47, 44), 'buy'), (5.09e-06, 6193.11, 1)),
 ((datetime.datetime(2018, 6, 25, 10, 48), 'buy'), (0.10248, 12371.01, 2))]

In [17]:
# Sacamos los diferentes tipos de órdenes
dataParsed.map(lambda x: (x.type, x.reason, x.order_type)).distinct().take(10)

[('done', 'canceled', None),
 ('open', None, None),
 ('received', None, 'limit'),
 ('match', None, None),
 ('done', 'filled', None),
 ('heartbeat', None, None),
 ('received', None, 'market')]

In [18]:
# Al no hacer sample debe haber alguna fila al final que me de problemas con el parseo
# data = spark.read.json(data_file)
# dataParsed = data.rdd.map(lambda x: parseaditoBomba(x))
# dataBTC_USD=dataParsed.filter(lambda x: x.product_id == "BTC-USD")
# dataBTC_USD=dataBTC_USD.filter(lambda x: x.type == "match" and x.size != None and x.time.day != None).map(lambda x: (x.time, (x.size,1)))
# dataBTC_USD.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).take(3)

### PARTE 2 - Añadiendo precio máximo y minimo

In [19]:
dataBTC_USD_match.take(5)[0][1][1]

6194.11

In [14]:
dataBTC_USD_matchPrice = dataBTC_USD.filter(lambda x: x.type == "match" and x.size != None and x.time.day != None).map(lambda x: (x.time,x.side, x.price))
dataBTC_USD_matchPrice.take(5)

[(datetime.datetime(2018, 6, 25, 10, 45, 4), 'buy', 6194.11),
 (datetime.datetime(2018, 6, 25, 10, 45, 28), 'sell', 6194.14),
 (datetime.datetime(2018, 6, 25, 10, 45, 28), 'sell', 6194.24),
 (datetime.datetime(2018, 6, 25, 10, 47, 27), 'sell', 6193.62),
 (datetime.datetime(2018, 6, 25, 10, 47, 44), 'buy', 6193.11)]

In [15]:
import pyspark.sql.functions as F
DF_dataBTC_USD_matchPrice=dataBTC_USD_matchPrice.toDF(["time", "side","price"])
DF_dataBTC_USD_matchMaxPrice=DF_dataBTC_USD_matchPrice.groupBy("time", "side").agg(F.max("price").alias("max")).rdd.map(lambda x: ((x.time, x.side),x.max))
DF_dataBTC_USD_matchMinPrice=DF_dataBTC_USD_matchPrice.groupBy("time", "side").agg(F.min("price").alias("min")).rdd.map(lambda x: ((x.time, x.side),x.min))

In [22]:
DF_dataBTC_USD_matchPrice.take(3)

[Row(time=datetime.datetime(2018, 6, 25, 10, 45, 4), side='buy', price=6194.11),
 Row(time=datetime.datetime(2018, 6, 25, 10, 45, 28), side='sell', price=6194.14),
 Row(time=datetime.datetime(2018, 6, 25, 10, 45, 28), side='sell', price=6194.24)]

In [16]:
dataBTC_USD_matchReduced=dataBTC_USD_match.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1], x[2]+y[2]))
dataBTC_USD_matchReducedMax=dataBTC_USD_matchReduced.leftOuterJoin(DF_dataBTC_USD_matchMaxPrice)
dataBTC_USD_matchReducedDEF=dataBTC_USD_matchReducedMax.leftOuterJoin(DF_dataBTC_USD_matchMinPrice)

In [ ]:
dataBTC_USD_matchReducedDEF.take(30)

In [ ]:
dataBTC_USD_matchReducedDEF.count()

In [ ]:
dataBTC_USD_matchReduced.count()

In [17]:
dataBTC_USD_matchReducedDEF.persist()

PythonRDD[64] at RDD at PythonRDD.scala:49

In [18]:
dataBTC_USD_matchReducedDEFformated=dataBTC_USD_matchReducedDEF.mapValues(lambda x: x[0] + (x[1], )).map(lambda x: (x[0][0],x[0][1],x[1][0][0],x[1][0][1]/x[1][0][2],x[1][0][2],x[1][1],x[1][2]))
dataBTC_USD_matchReducedDEFformated.take(5)

[(datetime.datetime(2018, 6, 25, 15, 41, 41),
  'sell',
  0.001,
  6290.0,
  1,
  6290.0,
  6290.0),
 (datetime.datetime(2018, 6, 25, 16, 12, 53),
  'sell',
  0.001,
  6295.56,
  1,
  6295.56,
  6295.56),
 (datetime.datetime(2018, 6, 25, 15, 43, 13),
  'sell',
  0.001,
  6290.0,
  1,
  6290.0,
  6290.0),
 (datetime.datetime(2018, 6, 25, 16, 14, 11),
  'buy',
  0.001,
  6295.53,
  1,
  6295.53,
  6295.53),
 (datetime.datetime(2018, 6, 25, 12, 20, 22),
  'buy',
  1.04205613,
  6156.53,
  1,
  6156.53,
  6156.53)]

In [19]:
DFdataBTC_USD_matchReducedDEF=dataBTC_USD_matchReducedDEFformated.toDF(["time","maker_side","size", "price_mean", "num_orders","price_max","price_min"])

In [20]:
#DFdataBTC_USD_matchReducedDEF.write.option("header", "true").csv("../Desktop/Data Science/00 -TFM/8- Fichero de prueba/BTC_USD_dataInput.csv")

In [21]:
from pyspark.sql.functions import col
DFdataBTC_USD_matchReducedDEFSorted=DFdataBTC_USD_matchReducedDEF.sort(col("time").desc())
DFdataBTC_USD_matchReducedDEFSorted.take(5)

[Row(time=datetime.datetime(2018, 6, 25, 18, 31, 46), maker_side='sell', size=0.00185, price_mean=6278.75, num_orders=1, price_max=6278.75, price_min=6278.75),
 Row(time=datetime.datetime(2018, 6, 25, 18, 31, 34), maker_side='sell', size=0.1803, price_mean=6274.35, num_orders=1, price_max=6274.35, price_min=6274.35),
 Row(time=datetime.datetime(2018, 6, 25, 18, 30, 31), maker_side='sell', size=0.5208437, price_mean=6274.35, num_orders=4, price_max=6274.35, price_min=6274.35),
 Row(time=datetime.datetime(2018, 6, 25, 18, 29, 18), maker_side='buy', size=0.6315, price_mean=6276.0650000000005, num_orders=2, price_max=6277.12, price_min=6275.01),
 Row(time=datetime.datetime(2018, 6, 25, 18, 28, 46), maker_side='sell', size=0.0251, price_mean=6277.29, num_orders=1, price_max=6277.29, price_min=6277.29)]

In [22]:
DFdataBTC_USD_matchReducedDEFSorted.persist()

DataFrame[time: timestamp, maker_side: string, size: double, price_mean: double, num_orders: bigint, price_max: double, price_min: double]

In [50]:
a=DFdataBTC_USD_matchReducedDEFSorted.rdd.map(lambda x : (str(x[0]),x[1],x[2],x[3],x[4],x[5],x[6]))
b=a.toDF()
c=b.toPandas()


## PARTE 3 - GAP OFERTA DEMANDA

In [24]:
dataParsed.filter(lambda x: x.type == "hertbreath").take(5)

[]

In [ ]:
import os 
os.environ.get('TZ', 'dateutil/:')
#DFdataBTC_USD_matchReducedDEF.toPandas(9)